In [63]:
#!pip install nltk

In [64]:
#!conda install -y -c conda-forge 'sqlalchemy>=2.0.4'

In [65]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])


[nltk_data] Downloading package punkt to /home/brownlouis/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/brownlouis/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/brownlouis/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [66]:
import re
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin, ClassifierMixin
from sklearn.feature_extraction.text import  FeatureHasher,CountVectorizer, TfidfTransformer
from sklearn.metrics import accuracy_score

In [67]:
from sqlalchemy import create_engine
# load data from database
engine = create_engine('sqlite:///pipeline_proj.db')
with engine.begin() as conn:
  df = pd.read_sql_table('messages_categories',conn)
X = df.iloc[:,:2]
y = df.iloc[:,2:]

X.drop('id',  axis=1, inplace=True)



In [68]:
import os
import pickle

def load_or_run(filename):
    def decorator(func):
        def wrapper(*args, **kwargs):
            if os.path.exists(filename):
                with open(filename, 'rb') as f:
                    return pickle.load(f)
            else:
                result = func(*args, **kwargs)
                with open(filename, 'wb') as f:
                    pickle.dump(result, f)
                return result
        return wrapper
    return decorator


In [69]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

### 2. Write a tokenization function to process your text data

In [70]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
# Download required datasets
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

def tokenize(text):
    

    text = str(text)
    
    # Convert to lowercase
    text = text.lower()
    
    # Remove punctuation
    text = ''.join([char for char in text if char.isalnum() or char.isspace()])
    
    # Tokenize
    tokens = word_tokenize(text)
    
    STOPWORDS = set(stopwords.words('english'))
    # Remove stopwords
    tokens = [word for word in tokens if word not in STOPWORDS]
    
    # Lemmatize
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    return tokens


[nltk_data] Downloading package punkt to /home/brownlouis/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/brownlouis/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/brownlouis/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [71]:
class ShapeChecker(BaseEstimator, TransformerMixin):
    def __init__(self, name=""):
        self.name = name

    def fit(self, X, y=None):
        print(f"Shape of X at '{self.name}': {X.shape}")
        if y is not None:
            print(f"Shape of y at '{self.name}': {y.shape}")
        return self

    def transform(self, X):
        # We don't modify X in any way.
        return X

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [72]:
pipeline = Pipeline([
  #  ('shape_before_vect', ShapeChecker(name="Before Vectorization")),
    ('vect', CountVectorizer(tokenizer=tokenize)),
  #  ('shape_after_vect', ShapeChecker(name="After Vectorization")),
    ('tfidf', TfidfTransformer()),
  #  ('shape_after_tfidf', ShapeChecker(name="After TF-IDF")),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [73]:
X_train = X_train.squeeze() 

In [74]:
# Train the pipeline
@load_or_run('models/model_1.pkl')
def train_model_1(model=pipeline,Xs=X_train, ys=y_train):
    pi= model.fit(Xs, ys)
    return pi

pipl = train_model_1(model=pipeline,Xs=X_train, ys=y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [75]:
from sklearn.metrics import classification_report

In [76]:
X_test = X_test.squeeze()
X_test.to_csv("test.csv")
y_pred = pipl.predict(X_test)


reports = {}

for idx, column in enumerate(y_test.columns):
    reports[column] = classification_report(y_test[column], y_pred[:, idx])

/home/brownlouis/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/brownlouis/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/brownlouis/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/brownlouis/.local/lib/

In [77]:
def extract_metrics(report):
    # Assume that the desired metrics line is the last one in the report
    metrics_line = report.strip().split('\n')[-1]
    
    # Extract values using regex
    values = re.findall(r"(\d+\.\d+|\d+)", metrics_line)
    
    if len(values) != 4:
        raise ValueError(f"Expected 4 values, but found {len(values)}")
    
    precision, recall, f1_score, support = map(float, values)
    return precision, recall, f1_score, support

categories = []
precision_vals = []
recall_vals = []
f1_score_vals = []
support_vals = []

for category, report in reports.items():
    try:
        precision, recall, f1_score, support = extract_metrics(report)
        categories.append(category)
        precision_vals.append(precision)
        recall_vals.append(recall)
        f1_score_vals.append(f1_score)
        support_vals.append(support)
    except ValueError as e:
        print(f"Error for category {category}: {e}")
        
# Create a dataframe from the lists
df = pd.DataFrame({
    'Category': categories,
    'Precision': precision_vals,
    'Recall': recall_vals,
    'F1-Score': f1_score_vals,
    'Support': support_vals
})

print(df)

                  Category  Precision  Recall  F1-Score  Support
0                  related       0.66    0.75      0.67   7863.0
1                  request       0.77    0.83      0.77   7863.0
2                    offer       0.99    1.00      0.99   7863.0
3              aid_related       0.53    0.57      0.53   7863.0
4             medical_help       0.86    0.92      0.88   7863.0
5         medical_products       0.92    0.95      0.93   7863.0
6        search_and_rescue       0.95    0.97      0.96   7863.0
7                 security       0.96    0.98      0.97   7863.0
8                 military       0.95    0.97      0.95   7863.0
9              child_alone       1.00    1.00      1.00   7863.0
10                   water       0.89    0.94      0.91   7863.0
11                    food       0.81    0.89      0.84   7863.0
12                 shelter       0.83    0.91      0.87   7863.0
13                clothing       0.97    0.98      0.98   7863.0
14                   mone

In [78]:
accuracy = (y_pred == y_test).mean()
accuracy

related                   0.746789
request                   0.829454
offer                     0.995676
aid_related               0.565687
medical_help              0.918988
medical_products          0.951164
search_and_rescue         0.973038
security                  0.979143
military                  0.966934
child_alone               1.000000
water                     0.937556
food                      0.890373
shelter                   0.906016
clothing                  0.984230
money                     0.976218
missing_people            0.987409
refugees                  0.960956
death                     0.952944
other_aid                 0.861249
infrastructure_related    0.932977
transport                 0.952054
buildings                 0.945059
electricity               0.979524
tools                     0.994659
hospitals                 0.988681
shops                     0.995167
aid_centers               0.987791
other_infrastructure      0.954725
weather_related     

### 6. Improve your model
Use grid search to find better parameters. 

In [79]:
parameters = {
    'clf__estimator__n_estimators': [100, 150, 200, 250,300],
    'clf__estimator__min_samples_split': [2, 3, 4, 8, 10],
    'clf__estimator__max_depth': [100, 110, 150,None],
    'vect__ngram_range': ((1, 1), (1, 2)),
    'vect__max_df': (0.5, 0.75, 1.0),
    'vect__max_features': (None, 2500, 5000),
    'tfidf__use_idf': (True, False)
}

# create grid search object
cv =  RandomizedSearchCV(pipeline, param_distributions=parameters, cv=3, verbose=2, random_state=42)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [80]:
# Train the pipeline
@load_or_run('models/model_2.pkl')
def train_model_2(model=cv,Xs=X_train, ys=y_train):
    fitted = model.fit(Xs, ys)
    return fitted

fitted = train_model_2(model=cv,Xs=X_train, ys=y_train)

In [81]:
fitted.best_params_

{'vect__ngram_range': (1, 1),
 'vect__max_features': None,
 'vect__max_df': 0.5,
 'tfidf__use_idf': False,
 'clf__estimator__n_estimators': 100,
 'clf__estimator__min_samples_split': 10,
 'clf__estimator__max_depth': 110}

In [82]:
pred_cv = fitted.best_estimator_.predict(X_test)

In [83]:
reports2 = {}

for idx, column in enumerate(y_test.columns):
    reports2[column] = classification_report(y_test[column], pred_cv[:, idx])

/home/brownlouis/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/brownlouis/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/brownlouis/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/brownlouis/.local/lib/

In [84]:
categories2 = []
precision_vals2 = []
recall_vals2 = []
f1_score_vals2 = []
support_vals2 = []

for category, report in reports2.items():
    try:
        precision, recall, f1_score, support = extract_metrics(report)
        categories2.append(category)
        precision_vals2.append(precision)
        recall_vals2.append(recall)
        f1_score_vals2.append(f1_score)
        support_vals2.append(support)
    except ValueError as e:
        print(f"Error for category {category}: {e}")
        
# Create a dataframe from the lists
df2 = pd.DataFrame({
    'Category': categories,
    'Precision': precision_vals,
    'Recall': recall_vals,
    'F1-Score': f1_score_vals,
    'Support': support_vals
})

print(df2)

                  Category  Precision  Recall  F1-Score  Support
0                  related       0.66    0.75      0.67   7863.0
1                  request       0.77    0.83      0.77   7863.0
2                    offer       0.99    1.00      0.99   7863.0
3              aid_related       0.53    0.57      0.53   7863.0
4             medical_help       0.86    0.92      0.88   7863.0
5         medical_products       0.92    0.95      0.93   7863.0
6        search_and_rescue       0.95    0.97      0.96   7863.0
7                 security       0.96    0.98      0.97   7863.0
8                 military       0.95    0.97      0.95   7863.0
9              child_alone       1.00    1.00      1.00   7863.0
10                   water       0.89    0.94      0.91   7863.0
11                    food       0.81    0.89      0.84   7863.0
12                 shelter       0.83    0.91      0.87   7863.0
13                clothing       0.97    0.98      0.98   7863.0
14                   mone

In [85]:
are_values_equal = (df.values == df2.values).all()
print(are_values_equal)

True


In [86]:
accuracy = (pred_cv == y_test).mean()
accuracy

related                   0.756963
request                   0.833270
offer                     0.995803
aid_related               0.587562
medical_help              0.922294
medical_products          0.952308
search_and_rescue         0.973293
security                  0.979524
military                  0.967061
child_alone               1.000000
water                     0.939718
food                      0.894061
shelter                   0.910085
clothing                  0.984230
money                     0.976599
missing_people            0.987409
refugees                  0.961338
death                     0.954597
other_aid                 0.873076
infrastructure_related    0.934631
transport                 0.952435
buildings                 0.947984
electricity               0.979779
tools                     0.994659
hospitals                 0.988681
shops                     0.995167
aid_centers               0.987791
other_infrastructure      0.955869
weather_related     

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [87]:
class TextToDictTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        transformed = []
        for text in X:
            word_counts = {}
            for word in text.split():
                word_counts[word] = word_counts.get(word, 0) + 1
            transformed.append(word_counts)
        return transformed

In [88]:
!conda update -y conda

/bin/bash: line 1: conda: command not found


In [89]:
#!pip install transformers

In [90]:
#!pip install huggingface_hub

In [91]:
#!conda install -y pytorch torchvision torchaudio -c pytorch 

In [92]:
from transformers import BertTokenizer, BertModel

In [93]:
import torch

In [94]:
from transformers import BertTokenizer, BertModel
import torch
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")

class BertEmbedder(BaseEstimator, TransformerMixin):
    def __init__(self, tokenizer, model):
        self.tokenizer = tokenizer
        self.model = model

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        if isinstance(X, (pd.Series, pd.DataFrame)):
            X = X.tolist()
        inputs = self.tokenizer(X, return_tensors="pt", padding=True, truncation=True, max_length=100)
        with torch.no_grad():
            embeddings = self.model(**inputs).last_hidden_state[:, 0, :].numpy()
        return embeddings


In [95]:
svm_clf = SVC(probability=True)

In [96]:
from catboost import CatBoostClassifier

In [97]:
from lightgbm import LGBMClassifier

In [98]:
class MultiOutputTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, *args, **kwargs):
        self.moc = MultiOutputClassifier(RandomForestClassifier(*args,**kwargs))
    
    def fit(self, X, y):
        self.moc.fit(X, y)
        return self
    
    def transform(self, X):
        # This would produce an array of shape [n_samples, n_outputs]
        # where n_outputs is the number of output variables you have
        return self.moc.predict(X)


In [99]:
# Pipelines for text processing
text_pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize, ngram_range=(1, 1), max_features=None, max_df=0.5)),
    ('tfidf', TfidfTransformer(use_idf=False))
])

hashing_pipeline = Pipeline([
    ('text_to_dict', TextToDictTransformer()),
    ('hasher', FeatureHasher(n_features=10, input_type='dict'))
])

bert_pipeline = Pipeline([
    ('bert_embedder', BertEmbedder(tokenizer, model))
])


union = FeatureUnion([
    ('text', text_pipeline),
    ('hasher', hashing_pipeline)
  #  ('bert', bert_pipeline)
])

# Then, incorporate this into a pipeline
multi_output_transformer_pipeline = Pipeline([
    ('features', union),
    ('moc_transformer', MultiOutputTransformer())
])

# And finally, add it to your FeatureUnion
new_union = FeatureUnion([
    ('text', text_pipeline),
    ('hasher', hashing_pipeline),
    # ('bert', bert_pipeline),
    ('moc_transformer', multi_output_transformer_pipeline)
], n_jobs=-1)

# Create pipelines for each individual classifier
rf_pipeline = Pipeline([
    ('features',new_union),
    ('rf',MultiOutputClassifier(RandomForestClassifier(n_jobs=-1, n_estimators=100, min_samples_split=10, max_depth=110)))
])

lr_pipeline = Pipeline([
    ('features',new_union),
    ('scaler', StandardScaler(with_mean=False)), 
    ('lr', MultiOutputClassifier(LogisticRegression(solver='saga', n_jobs=-1)))
])

# Adding the Gradient Boosted Classifier pipeline
gb_pipeline = Pipeline([
    ('features', new_union),
    ('gb', MultiOutputClassifier(GradientBoostingClassifier()))
])

cat_pipeline = Pipeline([
    ('features', new_union),
    ('scaler', StandardScaler(with_mean=False)),  # Step 1: Standardize the data
    ('classifier', MultiOutputClassifier(CatBoostClassifier(verbose=0,thread_count=-1)))  # Step 2: Use CatBoost as the classifier
])

# Addding SVM Classifier pipeline
svm_pipeline = Pipeline([
    ('features', new_union),
    ('scaler', StandardScaler(with_mean=False)),  # Use with_mean=False for sparse matrices
    ('svm', MultiOutputClassifier(svm_clf))
])

# Create the pipeline
lgbm_pipeline = Pipeline([
    ('features', new_union),
    ('scaler', StandardScaler(with_mean=False)),  # Step 1: Standardize the data
    ('classifier', MultiOutputClassifier(LGBMClassifier(n_jobs=-1,force_col_wise=True)))  # Step 2: Use LightGBM as the classifier
])


In [100]:
y_train = y_train.drop(columns=['child_alone']) # only one level occurs in the dataset

In [101]:
y_test = y_test.drop(columns=['child_alone'])

In [102]:
from sklearn.base import BaseEstimator, ClassifierMixin
from joblib import Parallel, delayed
import numpy as np

def fit_estimator(name, estimator, X, y):
    estimator.fit(X, y)
    return name, estimator

def predict_estimator(estimator, X):
    return estimator.predict_proba(X)

class MultiOutputVotingClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, estimators, voting='soft'):
        self.estimators = {name: estimator for name, estimator in estimators}
        self.voting = voting

    def fit(self, X, y):
        self.estimators = dict(
            Parallel(n_jobs=-1, backend="threading")(
                delayed(fit_estimator)(name, estimator, X, y) for name, estimator in self.estimators.items()
            )
        )
        return self

    def predict(self, X):
        if self.voting == 'soft':
            all_probas = Parallel(n_jobs=-1, backend="threading")(
                delayed(predict_estimator)(estimator, X) for _, estimator in self.estimators.items()
            )
            avg_probas = [np.mean([probas[i] for probas in all_probas], axis=0) for i in range(len(all_probas[0]))]
            predictions = np.array([np.argmax(proba, axis=1) for proba in avg_probas]).T
        else:
            preds = Parallel(n_jobs=-1, backend="threading")(
                delayed(predict_estimator)(estimator, X) for _, estimator in self.estimators.items()
            )
            predictions = np.round(np.mean(preds, axis=0))

        return predictions


ensemble = MultiOutputVotingClassifier(estimators=[
    ('rf_pipeline', rf_pipeline),
    ('lr_pipeline', lr_pipeline),
    ('gb_pipeline', gb_pipeline),
    ('cat_pipeline',cat_pipeline),
    ('lgbm_pipeline', lgbm_pipeline)
], voting='soft')
    
#ensemble.fit(X_train, y_train)
#predictions = ensemble.predict(X_test)

In [103]:
# Creating the parameter grid
parameters = {
    'rf__estimator__n_estimators': [50, 100, 200],
    'rf__estimator__min_samples_split': [2, 3, 4],
    
    'lr__estimator__C': [0.1, 1, 10],
    'lr__estimator__penalty': ['l1', 'l2'],
    
    'gb__estimator__n_estimators': [50, 100, 150],
    'gb__estimator__learning_rate': [0.01, 0.05, 0.1],
    'gb__estimator__subsample': [0.8, 0.9, 1]
    # If you also want to tune FeatureHasher
    #'features__hasher__hasher__n_features': [10, 20, 30]
}

# Create grid search object
from sklearn.model_selection import KFold
kf = KFold(n_splits=3, shuffle=True, random_state=42)
cv = RandomizedSearchCV(ensemble, param_distributions=parameters,  cv=kf, verbose=2)

In [104]:
# Generate sample weights; for demonstration, class 1 is the minority class
sample_weight = [1 if label == 1 else 0.5 for label in y_train]

In [105]:
# Train the pipeline
@load_or_run('models/model_4.pkl')
def train_model_3(model=cv,Xs=X_train, ys=y_train):
    trained = model.fit(Xs, ys)
    return trained

In [106]:
fit_3 = train_model_3(model=ensemble,Xs=X_train, ys=y_train) 

In [107]:
predictions = fit_3.predict(X_test)

KeyboardInterrupt: 

: 

In [ ]:
reports2 = {}
for idx, column in enumerate(y_test.columns):
    reports2[column] = classification_report(y_test[column], predictions[:, idx])

/home/brownlouis/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/brownlouis/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/brownlouis/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/brownlouis/.local/lib/

In [ ]:
categories = []
precision_vals = []
recall_vals = []
f1_score_vals = []
support_vals = []

for category, report in reports2.items():
    try:
        precision, recall, f1_score, support = extract_metrics(report)
        categories.append(category)
        precision_vals.append(precision)
        recall_vals.append(recall)
        f1_score_vals.append(f1_score)
        support_vals.append(support)
    except ValueError as e:
        print(f"Error for category {category}: {e}")
        
# Create a dataframe from the lists
df3 = pd.DataFrame({
    'Category': categories,
    'Precision': precision_vals,
    'Recall': recall_vals,
    'F1-Score': f1_score_vals,
    'Support': support_vals
})

print(df3)

                  Category  Precision  Recall  F1-Score  Support
0                  related       0.68    0.75      0.67   7863.0
1                  request       0.77    0.83      0.77   7863.0
2                    offer       0.99    1.00      0.99   7863.0
3              aid_related       0.55    0.58      0.52   7863.0
4             medical_help       0.86    0.92      0.88   7863.0
5         medical_products       0.91    0.95      0.93   7863.0
6        search_and_rescue       0.95    0.97      0.96   7863.0
7                 security       0.96    0.98      0.97   7863.0
8                 military       0.94    0.97      0.95   7863.0
9                    water       0.88    0.94      0.91   7863.0
10                    food       0.82    0.89      0.84   7863.0
11                 shelter       0.83    0.91      0.87   7863.0
12                clothing       0.97    0.98      0.98   7863.0
13                   money       0.96    0.98      0.96   7863.0
14          missing_peopl

In [ ]:
accuracy = (predictions == y_test).mean()
accuracy

related                   0.754674
request                   0.831744
offer                     0.995930
aid_related               0.582984
medical_help              0.919878
medical_products          0.951291
search_and_rescue         0.973038
security                  0.979270
military                  0.966679
water                     0.937683
food                      0.892789
shelter                   0.906397
clothing                  0.984103
money                     0.975836
missing_people            0.987409
refugees                  0.961211
death                     0.953071
other_aid                 0.867735
infrastructure_related    0.931833
transport                 0.952181
buildings                 0.946712
electricity               0.979779
tools                     0.994659
hospitals                 0.988681
shops                     0.995167
aid_centers               0.987664
other_infrastructure      0.954216
weather_related           0.738268
floods              

### 9. Export your model as a pickle file

In [ ]:
import pickle
def save_model(model, model_filepath):
    """ Exports the final model as a pickle file."""
    pickle.dump(model, open(model_filepath, 'wb'))


### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [ ]:
df.to_csv("table1.csv")
df2.to_csv("table2.csv")
df3.to_csv("table4.csv")      